In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
import scanpy as sc 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches

import sys 
sys.path.append('../../src')

/ihome/djishnu/alw399/.local/lib/python3.9/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [18]:
from spaceoracle.plotting.cartography import Cartography


In [19]:
ref_adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data_2025/mouse_kidney_13.h5ad')
test_adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data_2025/mouse_kidney_14.h5ad')

ref_models_dir = '/ix/djishnu/shared/djishnu_kor11/lasso_runs/mouse_kidney_13'

In [20]:
ref_adata

AnnData object with n_obs × n_vars = 8509 × 3058
    obs: 'ct1', 'ct2', 'ct3', 'ct4', 'cond', 'medulla_cortex', 'domain', 'cell_type', 'cell_type_int'
    uns: 'cell_thresholds', 'cell_type_colors', 'neighbors', 'pca', 'received_ligands', 'received_ligands_tfl', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial', 'spatial_unscaled'
    varm: 'PCs'
    layers: 'imputed_count', 'normalized_count', 'raw_count'
    obsp: 'connectivities', 'distances'

In [21]:
test_adata

AnnData object with n_obs × n_vars = 6102 × 3058
    obs: 'batch', 'rxn', 'n_counts', 'ct1', 'ct2', 'ct3', 'high_q', 'ct4', 'chip', 'exp', 'cond', 'samp-id', 'samp-type', 'cell_type', 'cell_type_int', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'log1p'
    obsm: 'spatial', 'spatial_unscaled'
    layers: 'imputed_count', 'normalized_count', 'raw_count'
    obsp: 'connectivities', 'distances'

In [22]:
matching = pd.read_csv('matching_13_14.csv', index_col=0)
matching

,reference_cell
0,
AAACCCAAGCCATTCA-1,CGTCCATTCTCGAACA-13
AAACCCACAAATCAGA-1,ACTTATCAGAGCACTG-13
AAACCCACAACTCATG-1,TTCGCTGAGTCGCTAT-13
AAACCCAGTACAGAAT-1,GGTAATCCACGGAAGT-13
AAACCCAGTACGTGTT-1,ATTGTTCGTCACGACC-13
...,...
TTTGTTGAGCATATGA-2,CCGTTCACAGCGACCT-13
TTTGTTGGTGCCCACA-2,ACACTGAAGGCATCAG-13
TTTGTTGGTGGCCCAT-2,GGTCTGGCATGAGGGT-13


In [ ]:
np.all(test_adata.obs.index == matching.index)

True

In [ ]:
from spaceoracle.visionary import Visionary

import warnings
warnings.simplefilter("ignore", category=UserWarning)

renata = Visionary(
    ref_adata = ref_adata,
    test_adata = test_adata,
    ref_json_path = f'{ref_models_dir}/run_params.json',
    matching_annot='cell_type',
    prematching=matching
)


In [25]:
target = 'Mif' 
gex = 0

In [26]:
epithelial = np.where(test_adata.obs['cell_type'] == 'Epithelial')[0]
len(epithelial)

394

In [ ]:
renata.perturb(
    target, 
    n_propagation=4, 
    save_layer=True,
    cells = epithelial
)
gex_df = renata.adata.to_df(layer=f'{target}_4n_{gex}x')
gex_df.shape

In [ ]:
gex_df.to_parquet(f'/ix/djishnu/shared/djishnu_kor11/genome_screens/mouse_kidney_14/{target}_4n_{gex}x_epithelial.parquet')
